In [1]:
/* Add Deps */
%AddDeps edu.stanford.nlp stanford-corenlp 3.7.0
%AddDeps com.google.protobuf protobuf-java 2.6.1
%AddDeps com.databricks spark-csv_2.10 1.5.0 --transitive

// Non-repo dependencies 
%AddJar file:lib/corenlp-models.jar
%AddJar file:SE/target/scala-2.10/se_2.10-1.1.jar

Marking edu.stanford.nlp:stanford-corenlp:3.7.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps1635466249893929454/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps1635466249893929454/https/repo1.maven.org/maven2/edu/stanford/nlp/stanford-corenlp/3.7.0/stanford-corenlp-3.7.0.jar
Marking com.google.protobuf:protobuf-java:2.6.1 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps1635466249893929454/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps1635466249893929454/https/repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
Marking com.databricks:spark-csv_2.10:1.5.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps1635466249893929454/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps1635466249893929454/https/repo1.maven.org/maven2/org/apache/commons/commons-csv/1.1/commons-csv-1.1.jar
-> New file at /tmp/toree_add_deps1635466249893929454/https/repo1

In [2]:
import com.evan.kaggle.se.FeatureEngineering._
val sqlContext = org.apache.spark.sql.SQLContext.getOrCreate(sc)
import sqlContext.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import scala.collection.immutable.HashSet

In [14]:
val files = List("cooking")//, "crypto", "robotics", "biology", "travel", "diy")

/* Load and print all files */
val df_all = files.map(f => {
                        sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("../dat/"+f+"_clean.csv")
                   }).reduce(_ unionAll _).filter($"content" !== "")

In [15]:
val feat_udf = udf((t: String, c: String, ta: String) => 
    makeTrFeatures(3)(t, c, ta))

In [16]:
val flat = df_all.select(explode(feat_udf($"title", $"content", $"tags")))
val flat2 = flat.select($"col.*")

In [17]:
val s = System.nanoTime
println(s"There are ${flat2.count} rows")
val diff = (System.nanoTime - s)/1e9d
println(s"Elapsed: $diff seconds")

There are 4237705 rows
Elapsed: 699.17177708 seconds
